In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')

train=train.drop(['id'],axis=1)
#test=test.drop(['id'],axis=1)

In [3]:
train.columns[train.dtypes=='object']

Index(['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'],
      dtype='object')

In [4]:
train[train['Weight Capacity (kg)']=='Nan']

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price


In [5]:
train['Brand'].unique()

array(['Jansport', 'Under Armour', 'Nike', 'Adidas', 'Puma', nan],
      dtype=object)

In [6]:
train = train.replace(np.nan, "Na")


In [7]:
train['Weight Capacity (kg)']=train['Weight Capacity (kg)'].replace("Nan",np.nan)

In [8]:
import catboost
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [9]:
X=train.drop(['Price'],axis=1)
y=train['Price']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
# Initialize CatBoost Regressor
catboost_model = CatBoostRegressor(
    iterations=500,  # Number of boosting iterations
    learning_rate=0.1,  # Step size shrinkage
    depth=6,  # Maximum depth of trees
    loss_function='RMSE',  # Root Mean Squared Error for regression
    verbose=100,
    cat_features=['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color']# Print training progress every 100 iterations
)

# Train the model
catboost_model.fit(X_train, y_train)


0:	learn: 39.0230899	total: 262ms	remaining: 2m 10s
100:	learn: 38.9680446	total: 10.6s	remaining: 42.1s
200:	learn: 38.9264625	total: 23.4s	remaining: 34.9s
300:	learn: 38.8931232	total: 36.3s	remaining: 24s
400:	learn: 38.8648935	total: 49.6s	remaining: 12.3s
499:	learn: 38.8377325	total: 1m 3s	remaining: 0us


In [11]:
preds=catboost_model.predict(X_test)

In [12]:
mean_squared_error(preds,y_test,squared=False)

39.03374079744069

In [ ]:
test = test.replace(np.nan, "Na")

preds1=catboost_model.predict(test.drop(['id'],axis=1))

In [15]:
preds1

array([81.06875047, 82.53422753, 82.79295338, ..., 81.70742042,
       81.32063668, 80.81389181])

In [16]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [17]:
param_dist = {
    'iterations': [500, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0]
}

# Initialize CatBoost model
catboost_model = CatBoostRegressor(loss_function="RMSE", verbose=100,cat_features=['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'])

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    catboost_model, param_distributions=param_dist, n_iter=7, cv=3, scoring="neg_mean_squared_error", random_state=42
)
random_search.fit(X_train, y_train)

0:	learn: 39.0756462	total: 143ms	remaining: 2m 22s
100:	learn: 39.0034704	total: 9.83s	remaining: 1m 27s
200:	learn: 38.9706004	total: 19.1s	remaining: 1m 16s
300:	learn: 38.9091268	total: 32.1s	remaining: 1m 14s
400:	learn: 38.8542855	total: 44.5s	remaining: 1m 6s
500:	learn: 38.7939962	total: 57.9s	remaining: 57.7s
600:	learn: 38.7370079	total: 1m 10s	remaining: 46.8s
700:	learn: 38.6874658	total: 1m 23s	remaining: 35.4s
800:	learn: 38.6290362	total: 1m 36s	remaining: 24s
900:	learn: 38.5746738	total: 1m 49s	remaining: 12s
999:	learn: 38.5172738	total: 2m 2s	remaining: 0us
0:	learn: 38.9744479	total: 53.6ms	remaining: 53.6s
100:	learn: 38.9170991	total: 9.43s	remaining: 1m 23s
200:	learn: 38.8818853	total: 18.8s	remaining: 1m 14s
300:	learn: 38.8310383	total: 30.7s	remaining: 1m 11s
400:	learn: 38.7725494	total: 43.1s	remaining: 1m 4s
500:	learn: 38.7091410	total: 56.1s	remaining: 55.9s
600:	learn: 38.6518353	total: 1m 9s	remaining: 46.3s
700:	learn: 38.6000631	total: 1m 22s	remaini

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x794233a40c70>,
                   n_iter=7,
                   param_distributions={'depth': [4, 6, 8, 10],
                                        'iterations': [500, 1000],
                                        'l2_leaf_reg': [1, 3, 5, 7],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='neg_mean_squared_error')

In [18]:
 random_search.best_params_

{'subsample': 0.6,
 'learning_rate': 0.2,
 'l2_leaf_reg': 5,
 'iterations': 500,
 'depth': 4}

In [21]:
best_params = random_search.best_params_  # Or grid_search.best_params_

# Train the final model
final_model = CatBoostRegressor(**best_params, verbose=100,cat_features=['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'])
final_model.fit(X_train, y_train)

# Evaluate
y_pred = final_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Final Model RMSE: {rmse:.4f}")


0:	learn: 39.0216433	total: 78.7ms	remaining: 39.3s
100:	learn: 38.9693240	total: 6.69s	remaining: 26.4s
200:	learn: 38.9472977	total: 14.2s	remaining: 21.1s
300:	learn: 38.9281449	total: 21.6s	remaining: 14.3s
400:	learn: 38.9106874	total: 29s	remaining: 7.15s
499:	learn: 38.8947624	total: 36.3s	remaining: 0us
Final Model RMSE: 39.0351


In [22]:
test = test.replace(np.nan, "Na")

preds1=final_model.predict(test.drop(['id'],axis=1))

In [23]:
submission_df = pd.DataFrame({
    'id': test['id'],
    'Price':preds1
})

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

submission_df.head(5)

,id,Price
0,300000,81.187666
1,300001,82.229228
2,300002,82.608570
3,300003,81.410057
4,300004,77.968700
